In [14]:
import meshio
from matplotlib.colors import LinearSegmentedColormap, ListedColormap, BoundaryNorm
import matplotlib.pyplot as plt
import numpy as np
from xrd_simulator_addons import get_ground_truth_slice
from multiprocessing import Pool


config_file = 'xrd_simulator_addons/configs/config_crystal_exp4.yaml'

In [15]:
phase_map, crystal_map = get_ground_truth_slice.get_ground_truth(config = config_file)

# Base colormap (background=-1, classes 0..3)
cmap = ListedColormap([
    "#9e9e9e",  # background
    "#1f77b4",  # class 0
    "#ff7f0e",  # class 1
    "#2ca02c",  # class 2
])
bounds = [-1.5, -0.5, 0.5, 1.5, 2.5]
norm = BoundaryNorm(bounds, cmap.N)

# Make new colormaps: white → color
new_cmaps = []
for i, color in enumerate(cmap.colors):
    new_cmaps.append(
        LinearSegmentedColormap.from_list(f"class_{i}", ["white", color])
    )

fig, ax = plt.subplots(figsize=(6, 6))

im = ax.imshow(phase_map, origin='lower', interpolation='nearest',
               cmap=cmap, norm=norm)

# 2) Texture overlay where crystal_map==1 (keeps phase color, adds stripes)
ny, nx = crystal_map.shape
xx, yy = np.meshgrid(np.arange(nx), np.arange(ny))
# diagonal stripe pattern; tweak "8" (period) and "< 4" (duty cycle) to change look
stripe = ((xx + 1*yy) % 16) < 4
mask = (crystal_map != 0)

alpha = (mask & stripe).astype(float) * 0.65  # stripe opacity
overlay = np.zeros((ny, nx, 4), dtype=float)  # black stripes
overlay[..., 3] = alpha

ax.imshow(overlay, interpolation='nearest')

ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_title('z=0 slice (phase + crystal texture)')
cbar = plt.colorbar(im, ax=ax, ticks=[-1, 0, 1, 2], fraction=0.046, pad=0.04)
cbar.ax.set_yticklabels(['background', '0', '1', '2'])
plt.show()
plt.figure(figsize=(6,6))
for i in range(len(spec)):
    plt.plot(spec[i])
    print(spec[i].sum())
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'samples/mesh_crystal_exp4.pc'